In [14]:
import json
import os
import sys
import time

In [14]:
module_path = os.path.abspath(os.path.join('.'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

/Users/xmanatee/Desktop/alyona/analytics


https://oauth.vk.com/authorize?client_id=6052784&scope=271502&redirect_uri=https://oauth.vk.com/blank.html&display=page&v=5.63&response_type=token

In [ ]:
# copy token here
token = "b0eccfa8b4fe3dca107904cdcae88aee85291dfb37b141bf552fb8d625986dc3b0bbadf1384f4a9051abc"

In [ ]:
poll_group_id = "-172053584"

# VK API

In [ ]:
import vk
from vk.exceptions import VkAPIError

In [ ]:
session = vk.Session(token)
api = vk.API(session, v="5.74")

# Getting wall data

### Creating data dir if it doesn't exist

In [ ]:
os.makedirs("../data", exist_ok=True)

### Requesting first 100

In [ ]:
kwargs = {"owner_id": poll_group_id, "count": "100"}

group_wall_json = api.wall.get(**kwargs)

print("got {} items".format(len(group_wall_json["items"])))

### Requesting the rest

In [ ]:
while len(group_wall_json["items"]) < group_wall_json["count"]:
    kwargs.update({
        "offset": len(group_wall_json["items"])
    })
    extra_group_wall_json = api.wall.get(**kwargs)
    group_wall_json["items"].extend(extra_group_wall_json["items"])
    print("Number of items now: {}".format(len(group_wall_json["items"])))
    time.sleep(3)

### Saving the result

In [ ]:
with open("data/group_wall.json", "w") as f:
    json.dump(group_wall_json, f)

# Getting poll voters

In [ ]:
with open("data/group_wall.json", "r") as f:
    group_wall_json = json.load(f)

In [ ]:
def vote_in_poll(api, poll):
    answer_id = poll["answers"][0]["id"]
    kwargs = {
        "poll_id": poll["id"],
        "owner_id": poll["owner_id"],
        "answer_ids": [answer_id],
    }
    
    return api.polls.addVote(**kwargs)

In [ ]:
for i, wall_post in enumerate(group_wall_json["items"]):
    poll = get_poll(wall_post)
    if poll is None or poll["anonymous"]:
        continue
    poll_filename = "data/poll_{}.json".format(poll["id"])

    if os.path.isfile(poll_filename):
        continue
    
    if poll["answer_id"] == 0:
        print("needs answering: https://vk.com/asurveys?w=poll-172053584_{}".format(poll["id"]))
        continue

    kwargs = {
        "poll_id": poll["id"],
        "owner_id": poll["owner_id"],
        "count": 1000,
        "answer_ids": list(map(lambda answer: answer["id"], poll["answers"])),
        "fields": "nickname, screen_name, sex, photo, photo_medium, photo_big"
    }
    
    try:
        poll_voters = api.polls.getVoters(**kwargs)
    except VkAPIError as vk_error:
        print(vk_error)
        print(poll)
        time.sleep(1)
        continue
    with open(poll_filename, "w") as f:
        json.dump(poll_voters, f)
    if (i + 1) % 20 == 0:
        print("Processed {}/{} polls".format(i + 1, len(group_wall_json["items"])))
    time.sleep(5)